In [480]:
import pandas as pd 
from datetime import datetime
import openpyxl as px 
import psycopg2 as pg
import configparser
import numpy as np

In [481]:
hora_inicio = datetime.today()

In [482]:
crendenciais_path = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\18 - SQL\sets.conf'

In [483]:
#Senhas e credênciais 

credenciais = configparser.ConfigParser()

credenciais.read(crendenciais_path)

logins = credenciais['informacoes']


conn_dwh = pg.connect(
    user =logins['login'],
    password =logins['senha'],
    host = logins['host'],
    port = logins['port'],
    database = logins['data_base']
)

In [484]:

#Realizando conexão do banco -> cadastro histórico

cursor = conn_dwh.cursor()

#cursor.execute("select * from forestry.f_cfg_uso_solo where source = 'BSP' and data_reg > '2021-12-31' " )


#Se atentar as regiões de São paulo que são 1 e 2


#Necessário mudar a consulta para fazer um left join na consulta d_cfg_espacamento para trazer a informação de espaçamento do cadastro


cursor.execute("""
SELECT 
    f.data_plantio,
    f.cd_regiao,
    f.id_projeto,
    f.cd_projeto,
    f.cd_talhao,
    f.num_ciclo,
    f.cd_regime,
    f.num_rotacao,
    f.vlr_area,
    f.source,
    f.data_reg,
    f.est_reg,
    f.cd_uso_solo,
    f.cd_caracteristica_uso_solo,
    f.cd_espacamento,
    d.dcr_espacamento,
    p.nom_projeto
FROM 
    forestry.f_cfg_uso_solo f
LEFT JOIN 
    forestry.d_cfg_espacamento d
ON 
    f.cd_espacamento = d.cd_espacamento
    
LEFT JOIN 
    forestry.cfg_projeto p

ON
    f.id_projeto = p.id_projeto
    
WHERE 
    f.source = 'BSP' 
    AND p.source = 'BSP'
    AND p.cd_regiao in ('1','2')
    AND p.est_reg = 'A'
    AND f.tip_reg = 'A'  
    AND f.cd_regiao IN ('1','2')
    
    
    
""")



dados_consulta = cursor.fetchall()

cursor.close()

conn_dwh.close()

colunas = [desc[0] for desc in cursor.description]


cadastro_ciclo = pd.DataFrame(dados_consulta,columns=colunas)

In [485]:
df_boletim1 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2020_Boletim_da_Silvicultura.parquet'
df_boletim2 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2021_Boletim_da_Silvicultura.parquet'
df_boletim3 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2022_Boletim_da_Silvicultura.parquet'
df_boletim4 = r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2023_Boletim_da_Silvicultura.parquet'
df_boletim5 =r'F:\Planejamento_Florestal\Controles-Florestais\13-DB-real\Boletim da Silvicultura Daily\Anual\2024_Boletim_da_Silvicultura.parquet'

In [486]:
path_premissas = r'\\GLWFS02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\premissas.xlsx'

In [487]:
path_justificativas = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\justificativas.xlsx'

In [488]:
path_sequencia = r'\\glwfs02.lwart.net\LWC-FLORESTAL\Qualidade_Florestal\03- ADMINISTRATIVO\2023\06- COLABORADORES\Gabriel\19 - Projeto SOP\justificativas.xlsx'

In [489]:
path_recomendacao_adu = r'F:\Silvicultura\RECOMENDAÇÕES DE ADUBAÇÃO\RECOMENDAÇÕES_ADUBAÇÃO_SILVICULTURA_SP.xlsx'

path_rec_complementar = r'F:\Silvicultura\RECOMENDAÇÕES DE ADUBAÇÃO\RECOMENDAÇÕES_ADUBAÇÕES_COMPLEMENTARES_SP.xlsx'


In [490]:
sequencia_vigente = pd.read_excel(path_sequencia, sheet_name='Sequência de Operações', engine='openpyxl')

In [491]:
justificativas = pd.read_excel(path_justificativas,sheet_name='justificativas',engine='openpyxl')

In [492]:
def mascara(df, coluna,n_digitos):
    df[coluna] = df[coluna].astype(str)
    df[coluna] = str("000") + df[coluna]
    df[coluna] = df[coluna].astype(str).str[-n_digitos:] 
    return df

In [493]:
cadastro_ciclo = mascara(cadastro_ciclo,'id_projeto',4)
cadastro_ciclo = mascara(cadastro_ciclo,'cd_talhao',3)
cadastro_ciclo['objeto de locação'] = cadastro_ciclo['id_projeto'].astype(str) + cadastro_ciclo['cd_talhao'].astype(str)
cadastro_ciclo['vlr_area'] = cadastro_ciclo['vlr_area'].astype(float)
#Filtrando regimes 5 (Implantação) e 6 (Reforma) 
cadastro_ciclo = cadastro_ciclo[(cadastro_ciclo['cd_regime'] == 5) | (cadastro_ciclo['cd_regime']==6)]

#Filtrando somente o que é talhão para crescimento cd_caracteristica_uso_solo -> Não é possível filtrar o status. Caso contrário a regra do ciclo não irá funcionar

#Definindo vlr entrelinha 
cadastro_ciclo.loc[:,'vlr_entrelinha'] = cadastro_ciclo['dcr_espacamento'].str.split(' X ').str[0].str.replace(',','.').astype('float')
#Definindo vlr linha
cadastro_ciclo.loc[:,'vlr_linha'] = cadastro_ciclo['dcr_espacamento'].str.split(' X ').str[1].str[0].str.replace(',','.').astype('float')


In [494]:
def mes_operacional(df, coluna, nome_coluna):
    # Garantir que a coluna é do tipo datetime
    if not pd.api.types.is_datetime64_any_dtype(df[coluna]):
        df[coluna] = pd.to_datetime(df[coluna], errors='coerce')
    
    
    def calcular_mes_operacional(data):
        
        if pd.isna(data):
            return None 
        
        
        dia = data.day
        mes = data.month
        ano = data.year
        
        
        if dia > 20:
            mes = (mes % 12) + 1
            if mes == 1:
                ano += 1
        
        # Retornar um Timestamp, garantindo que todos são inteiros
        return pd.Timestamp(year=int(ano), month=int(mes), day=1)

    # Aplicar a função para a coluna
    df[nome_coluna] = df[coluna].apply(calcular_mes_operacional)

    return df
    

In [495]:
#Base de justificativas -> Talhões justificados para fitrarem o boletim

justificativas = mascara(justificativas,'Id projeto',4)
justificativas = mascara(justificativas,'Id talhão',3)
justificativas['objeto de locação'] = justificativas['Id projeto'].astype(str) + justificativas['Id talhão'].astype(str)

justificativas

,boletim,Id projeto,Id talhão,Boletim vigente justificativas,Observação,Classe de observação,objeto de locação
0,438023,0011,002,Não,Justificado,Classe 1,0011002
1,446988,0011,002,Não,NaN,NaN,0011002
2,165736,0015,013,Não,Justificado,Classe 1,0015013


In [496]:
#Data de Plantio origem cadastro histórico e ativo

data_de_plantio = cadastro_ciclo[['objeto de locação','data_plantio','cd_uso_solo']]

data_de_plantio = data_de_plantio.rename({'data_plantio':'Data de Plantio'},axis=1)

data_de_plantio = data_de_plantio[data_de_plantio ['Data de Plantio'].notna()]

data_de_plantio['Data de Plantio'] = pd.to_datetime(data_de_plantio['Data de Plantio'], dayfirst=True)

In [497]:
#Bases de premissas
insumos = pd.read_excel(path_premissas, sheet_name='insumos',engine = 'openpyxl')

operacao = pd.read_excel(path_premissas, sheet_name='operacao',engine = 'openpyxl')

regua_operacao = pd.read_excel(path_premissas, sheet_name='regua_operacao',engine = 'openpyxl')

tipo_aprovacao = pd.read_excel(path_premissas, sheet_name='status')

In [498]:
#Trabalhando as recomendações de adubações


recomendacao_adu = pd.read_excel(path_recomendacao_adu,sheet_name='Enviados_Silvicultura')

colunas_rec_adu = {
    recomendacao_adu.columns[1]: 'Projeto',
    recomendacao_adu.columns[3]: 'Talhão',
    recomendacao_adu.columns[6]: 'Insumo mecanizado',
    recomendacao_adu.columns[7]: 'Dose kg/ha calagem',
    recomendacao_adu.columns[8]: 'Insumo manual',
    recomendacao_adu.columns[9]: 'Dose kg/ha calagem 2',
    recomendacao_adu.columns[10]: 'Insumo base',
    recomendacao_adu.columns[11]: 'Dose kg/ha base',
    recomendacao_adu.columns[12]: 'Insumo cobertura',
    recomendacao_adu.columns[13]: 'Dose kg/ha cobertura',
    recomendacao_adu.columns[15]: 'Insumo cobertura 2',
    recomendacao_adu.columns[16]: 'Dose kg/ha cobertura 2',
    recomendacao_adu.columns[22]: 'Data de recomendação'
}

#Renomeando as colunas
recomendacao_adu = recomendacao_adu.rename(columns=colunas_rec_adu)

#Escolhendo colunas
recomendacao_adu = recomendacao_adu[['Projeto','Talhão','Insumo mecanizado','Dose kg/ha calagem','Insumo manual','Dose kg/ha calagem 2','Insumo base','Dose kg/ha base','Insumo cobertura','Dose kg/ha cobertura','Insumo cobertura 2','Dose kg/ha cobertura 2','Data de recomendação']]

#Removendo a primeira linha
recomendacao_adu = recomendacao_adu.iloc[1:]

#Aplicando máscara
recomendacao_adu = mascara(recomendacao_adu,'Projeto',4)
recomendacao_adu = mascara(recomendacao_adu,'Talhão',3)

#Coluna objeto de locação
recomendacao_adu['Objeto de locação'] = recomendacao_adu.apply(lambda x: x['Projeto'] + x['Talhão'], axis=1)

#Alterando o tipo de data
recomendacao_adu['Data de recomendação'] = pd.to_datetime(recomendacao_adu['Data de recomendação'])

#ordenando o data frame
recomendacao_adu = recomendacao_adu.sort_values(['Objeto de locação','Data de recomendação'], ascending=[True,False])

#Mantendo apenas os itens mais recentes

recomendacao_adu = recomendacao_adu.drop_duplicates('Objeto de locação',keep='first')

#Trazendo somente o que preciso no merge, sendo operações de base e removendo duplicatas em formulação
recomendacao_adu = recomendacao_adu.merge(
    insumos[insumos['Tipo de Adubação'] == 'Base'].drop_duplicates('Formulação')[['Formulação','Estratégia']] ,      
    left_on='Insumo base', right_on='Formulação', how='left').drop('Formulação',axis=1)

recomendacao_adu['Recomendação de Calagem'] = recomendacao_adu.apply(lambda x: x['Insumo manual'] if pd.isna(x['Insumo mecanizado']) else x['Insumo mecanizado'], axis=1 )
recomendacao_adu['Recomendação Dose kg/ha Calagem'] = recomendacao_adu.apply(lambda x: x['Dose kg/ha calagem 2'] if pd.isna(x['Dose kg/ha calagem']) else x['Dose kg/ha calagem'], axis=1 )

recomendacao_adu = recomendacao_adu.rename({'Estratégia': 'Estratégia Recomendada'}, axis=1)

#Inciando o processo de recomendação complementar

recomendacao_comp =  pd.read_excel(path_rec_complementar, sheet_name='Enviados Silvicultura')

colunas_comp = {
recomendacao_comp.columns[2] : 'Projeto',
recomendacao_comp.columns[4] : 'Talhão',
recomendacao_comp.columns[10] : 'Status',
recomendacao_comp.columns[11] : 'NPK complementar',
recomendacao_comp.columns[12] : 'Dose kg/ha complementar',
recomendacao_comp.columns[13] : 'Corretivos complementares',
recomendacao_comp.columns[14] : 'Dose kg/ha corretivos',
recomendacao_comp.columns[16] : 'Data de recomendação'
}
recomendacao_comp = recomendacao_comp.rename(columns=colunas_comp)

#Escolhendo colunas
recomendacao_comp = recomendacao_comp[['Projeto','Talhão','Status','NPK complementar','Dose kg/ha complementar','Corretivos complementares','Dose kg/ha corretivos','Data de recomendação']]

#Excluindo a primeira linha
recomendacao_comp = recomendacao_comp.iloc[1:]

recomendacao_comp['Data de recomendação'] = pd.to_datetime(recomendacao_comp['Data de recomendação'])


recomendacao_comp = mascara(recomendacao_comp,'Projeto',4)
recomendacao_comp = mascara(recomendacao_comp,'Talhão',3)
recomendacao_comp['Objeto de locação'] = recomendacao_comp.apply(lambda x: x['Projeto'] + x['Talhão'],axis=1)

#Ordenando o data frame
recomendacao_comp = recomendacao_comp.sort_values(['Objeto de locação','Data de recomendação'], ascending=[True, False])

#Exluindo duplicatas e mantendo somente o primeiro
recomendacao_comp = recomendacao_comp.drop_duplicates('Objeto de locação',keep='first')

#Filtrando somente o que tem recomendação realizada
recomendacao_comp = recomendacao_comp[recomendacao_comp['Status'] == 'Recomendação realizada']

#o que importa é que no nome tenha a palavra oxyfertil
recomendacao_comp['Corretivos complementares'] = recomendacao_comp['Corretivos complementares'].apply(lambda x: 'Oxyfertil' if 'Oxyfertil' in x else x)

#Excluindo onde todas as colunas são iguais a n.a ou nulas
recomendacao_comp = recomendacao_comp[((recomendacao_comp['NPK complementar'] != 'n.a.') & (pd.notna(recomendacao_comp['NPK complementar']))) | ((recomendacao_comp['Corretivos complementares'] != 'n.a.') & (pd.notna(recomendacao_comp['Corretivos complementares'])))]


In [499]:
#Ler o Boletim

boletim1 = pd.read_parquet(df_boletim1, engine='fastparquet')

boletim2 = pd.read_parquet(df_boletim2, engine='fastparquet')

boletim3 = pd.read_parquet(df_boletim3, engine='fastparquet')

boletim4 = pd.read_parquet(df_boletim4, engine='fastparquet')

boletim5 = pd.read_parquet(df_boletim5, engine='fastparquet')

boletim = pd.concat([boletim1,boletim2,boletim3,boletim4, boletim5], ignore_index=True)

#boletim = pd.concat([boletim3,boletim4, boletim5], ignore_index=True)

boletim = boletim[['ID_REGIAO','CD_BOLETIM_SILVICULTURA','DATA_OPERACAO','VLR_PRODUCAO','DCR_OPERACAO','CD_TALHAO','ID_PROJETO','CD_PROJETO','NOME_RECURSO','QUANTIDADE','UNIDADE_MED','DCR_FASE_PRESCRICAO','FLAG_OCORRENCIA','NOM_PROJETO','DCR_FASE_PRESCRICAO','TIP_APROVACAO']]

#Filtrando apenas São Paulo - IDREGIÃO == 1 e 2

boletim = boletim[ (boletim['ID_REGIAO'] == 1) | (boletim['ID_REGIAO'] ==2 )]

#Filtrando talhões que não são nulos -> Operações que não são necessárias apontamento de talhão -> Ex: Caminhão Munck
boletim_talhao_null = boletim['CD_TALHAO'].notna()
boletim = boletim[boletim_talhao_null]

boletim = mascara(boletim,'ID_PROJETO',4)
boletim = mascara(boletim, 'CD_TALHAO',3)

boletim['objeto de locação'] = boletim['ID_PROJETO'].astype(str) + boletim['CD_TALHAO'].astype(str)

In [500]:
#Cadastro e Ciclo

#Trazendo o ciclo do cadastro para o boletim

boletim = boletim.merge(cadastro_ciclo[['objeto de locação','nom_projeto','num_ciclo','num_rotacao','data_reg','cd_regime','cd_uso_solo','vlr_area','vlr_entrelinha','vlr_linha']], left_on='objeto de locação', right_on='objeto de locação', how='left')

#Filtrando apenas o que é regime de implantação e reforma
boletim = boletim[(boletim['cd_regime'] == 5) | (boletim['cd_regime'] == 6) ]


#Definindo o que é ciclo vigente
boletim['ciclo_vigente'] = boletim.apply(lambda x: 'Ciclo Vigente' if x['DATA_OPERACAO'] >= x['data_reg'] else 'Não Vigente',axis=1)

#Removendo os ciclos que não são vigentes

boletim = boletim[boletim['ciclo_vigente'] == 'Ciclo Vigente']

#Ordenando o boletim
boletim = boletim.sort_values(['objeto de locação','CD_BOLETIM_SILVICULTURA','data_reg'],ascending=[False,False,False])

#Removendo duplicatas para ter somente um único boletim
boletim = boletim.drop_duplicates(['objeto de locação','CD_BOLETIM_SILVICULTURA','NOME_RECURSO'],keep='first')

#Removendo cd_uso_solo vazios - Quando é removido o 'cd_uso_solo' vazio, garante que teremos apenas 
boletim = boletim[boletim['cd_uso_solo'].notna()]


In [501]:
#Retirando os boletins que tem justificativas

boletim = boletim.merge(justificativas[['boletim','Boletim vigente justificativas']],left_on='CD_BOLETIM_SILVICULTURA', right_on='boletim', how='left' ).drop('boletim',axis=1)

boletim = boletim[boletim['Boletim vigente justificativas'] != "Não"]

In [502]:
#Realizando merge para trazer as operações

boletim = boletim.merge(operacao[['Operações','Grupo de Operação','Operação Vigente','Tipo de operação']], left_on='DCR_OPERACAO', right_on='Operações', how='left').drop('Operações',axis=1)

#Filtrando DataFrame para remover todas as linhas nulas e manter apenas as linhas que tem operações cadastradas em premissas
linhas_nulas = boletim['Grupo de Operação'].notna()
boletim = boletim[linhas_nulas]


#Realizando merge para trazer os insumos

boletim = boletim.merge(insumos[['Insumo','Característica','Insumo Vigente','Fordor','Tipo de Adubação','Estratégia','Formulação']], left_on='NOME_RECURSO', right_on='Insumo', how='left').drop('Insumo',axis=1)

#Realizando merge para trazer as operações já conhecidas

boletim = boletim.merge(regua_operacao, left_on='DCR_OPERACAO', right_on='operação', how='left').drop('operação',axis=1)


#Status do boletim
boletim = boletim.merge(tipo_aprovacao[['Status','sigla','Status Vigente']], left_on='TIP_APROVACAO', right_on= 'sigla', how='left').drop('sigla',axis=1)

#Filtrando o status de acordo com o que está vigente em premissas

boletim = boletim[boletim['Status Vigente'] == 'Sim']


#Crianddo ID para encontrar a operação de plantio
boletim['id_plantio'] = boletim.apply(lambda x: 'Plantio' if x['Grupo de Operação'] == "Plantio" and x['Característica'] == 'Mudas' else None, axis=1)

In [503]:
# Percentual de área apontada
boletim["%realizado"] = boletim.apply(lambda x: x['VLR_PRODUCAO']/ x['vlr_area'] if pd.notna(x['vlr_area']) else x['VLR_PRODUCAO'],axis=1)

#Removendo talhões com operações menores que 20%
boletim = boletim[boletim['%realizado'] > 0.20]

#Filtrar operações e insumos vigentes
boletim = boletim[boletim['Operação Vigente'] == 'Sim']

boletim = boletim[boletim['Insumo Vigente'] == 'Sim']

#Ordenar por objeto de locação e por data (Ordem Decrescente)
boletim = boletim.sort_values(['objeto de locação','DATA_OPERACAO'], ascending=[True, False]) 

#Criar regra para remover duplicatas de boletins de plantio (Plantio com dois insumos -> Mudas Próprias e de terceiros)
duplicata_platio = (boletim['Grupo de Operação'] == 'Plantio')
#remover as duplicatas de acordo com a regra definida 
boletim.loc[duplicata_platio,:] = boletim.loc[duplicata_platio,:].drop_duplicates('CD_BOLETIM_SILVICULTURA')


In [504]:
#Código de plantio agrupado pelo ciclo

cod_plantio = boletim[boletim['Grupo de Operação'] == 'Plantio']
cod_plantio = cod_plantio.groupby('cd_uso_solo')['DATA_OPERACAO'].max().reset_index()

cod_plantio['Grupo de Operação'] = 'Plantio'
cod_plantio['cod_plantio'] = cod_plantio['cd_uso_solo'].astype(str) + cod_plantio['Grupo de Operação'].astype(str)

#Removendo talhões nulos
cod_plantio = cod_plantio[cod_plantio['cd_uso_solo'].notna()]



In [505]:
#Código de preapro agrupado pelo ciclo

cod_preparo = boletim[boletim['Grupo de Operação'] == 'Preparo']
cod_preparo = cod_preparo.groupby('cd_uso_solo')['DATA_OPERACAO'].max().reset_index()

cod_preparo['Grupo de Operação'] = 'Preparo'
cod_preparo['cod_preparo'] = cod_preparo['cd_uso_solo'].astype(str) + cod_preparo['Grupo de Operação']

In [506]:
#Realizando merge para trazer o campo cod_plantio

boletim = boletim.merge(cod_plantio, right_on=['cd_uso_solo', 'DATA_OPERACAO','Grupo de Operação'], left_on=['cd_uso_solo', 'DATA_OPERACAO','Grupo de Operação'], how= 'left')

boletim = boletim[boletim['cd_uso_solo'].notna()]

boletim = boletim.merge(cod_preparo, right_on=['cd_uso_solo','DATA_OPERACAO','Grupo de Operação'], left_on=['cd_uso_solo','DATA_OPERACAO','Grupo de Operação'], how='left')

boletim = boletim[boletim['cd_uso_solo'].notna()]

#Data de plantio de acordo com o cadastro
boletim = boletim.merge(data_de_plantio[['cd_uso_solo','Data de Plantio']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')

#Data  de preparo de acordo com o cod_preparo
boletim['Data de Preparo'] = boletim.apply(lambda x: x['DATA_OPERACAO'] if pd.notna(x['cod_preparo']) else None, axis=1)

In [507]:
#Replantio


#transformando o boletim em uma base exclusiva de replantio
replantio = boletim[boletim['QUANTIDADE'] > 0]

replantio = replantio[(replantio['Grupo de Operação'] == 'Replantio') & (replantio['Operação Vigente'] == 'Sim')]

replantio = replantio[(replantio['Característica'] == 'Mudas') & (replantio['Insumo Vigente'] == 'Sim')]

#Ordenando o data frame
replantio = replantio.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[False,True,False])
#Removendo as duplicatas para definir a ordem de operação
replantio_sduplicatas = replantio.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA',keep='first')

#Ordenando a base de sequência de operação para garantir que todos ficaram na mesma ordem

replantio_sduplicatas = replantio_sduplicatas.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[False,True])

replantio_sduplicatas['regra de mudança'] = (replantio_sduplicatas['CD_BOLETIM_SILVICULTURA'] != replantio_sduplicatas['CD_BOLETIM_SILVICULTURA'].shift(1)) & (replantio_sduplicatas['cd_uso_solo'] != replantio_sduplicatas['cd_uso_solo'].shift(1))

replantio_sduplicatas['grupos distintos'] = replantio_sduplicatas['regra de mudança'].cumsum()

replantio_sduplicatas['Sequência de operação'] = replantio_sduplicatas.groupby('grupos distintos').cumcount()+1


#levar a sequencia de operação de replantio para a base principal

replantio = replantio.merge(replantio_sduplicatas[['cd_uso_solo','CD_BOLETIM_SILVICULTURA','NOME_RECURSO','Sequência de operação']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','NOME_RECURSO'],how='left')


#Definindo o delta de plantio
replantio['Delta de plantio'] = replantio.apply(lambda x: (x['DATA_OPERACAO'] - x['Data de Plantio']).days if pd.notna(x['Data de Plantio']) else None, axis=1)

#Realizar o cálculo de mudas/ha e arredondar para cima
replantio['mudas/ha'] = np.ceil((replantio['QUANTIDADE'] / replantio['vlr_area']))

#mudas por m³
replantio['m3'] = replantio['vlr_entrelinha'] * replantio['vlr_linha']

#mudas/ha recomendado
replantio['mudas/ha recomendado'] = np.ceil((10000 / replantio['m3']))

#Replantio efetivo (total de mudas / mudas/ha recomendado)
replantio['Replantio Efetivo'] = round(replantio['mudas/ha'] / replantio['mudas/ha recomendado'],3)


#Criar a regra para definir qual é a primeira e a segunda operação
replantio['regra_operacao_replantio'] = np.where( replantio['Sequência de operação'] == 1 , '1º Replantio',
                                                 np.where( replantio['Sequência de operação'] == 2, '2º Replantio',
                                                        np.where( replantio['Sequência de operação'] == 3, '3º Replantio',                                              
                                                  replantio['Sequência de operação'].map(lambda x: f'{x:.0f}').astype(str) +'º Replantio'
                                                                )
                                                         )
                                                )

#Número de replantios no talhão
replantio['numero de replantios'] = replantio.groupby('cd_uso_solo')['Sequência de operação'].transform('max')


#Adicionar o export (Exportar todas as operações)
replantio.to_excel('Replantio.xlsx')

#Filtrar somente até segunda operação

replantio = replantio[(replantio['regra_operacao_replantio'] == '1º Replantio') | (replantio['regra_operacao_replantio'] == '2º Replantio') | (replantio['regra_operacao_replantio'] == '3º Replantio') ]

#Criando referência do boletim
replantio['Boletim Replantio'] = replantio['CD_BOLETIM_SILVICULTURA']

replantio = replantio.rename({'DATA_OPERACAO':'Data Replantio', 'Sequência de operação':'Replantio'},axis=1)


In [508]:
#Criando um data frame de dadas de preparo

data_de_preparo = boletim[['Data de Preparo','cd_uso_solo','objeto de locação','Status']]
data_de_preparo = data_de_preparo[data_de_preparo['Data de Preparo'].notna()].drop_duplicates('cd_uso_solo')


In [509]:
#Insumos fordor para a regra de pre plantio utilizando o fordor

#Agora pode apontar qualquer insumo até 5 depois do plantio 

#Apenas histórico, a regra mudou para qualquer insumo  com características de Pre

insumos_pre_plantio = boletim[['CD_BOLETIM_SILVICULTURA','NOME_RECURSO']]

insumos_pre_plantio = insumos_pre_plantio.merge(insumos[['Fordor','Insumo']], left_on='NOME_RECURSO', right_on='Insumo',how='left').drop('Insumo', axis=1)

insumos_pre_plantio = insumos_pre_plantio[insumos_pre_plantio['Fordor'] == 'Fordor']

In [510]:
# 1º Controle pré

id_controle_pre_plantio = boletim[(boletim['Grupo de Operação'] == 'Herbicida') | (boletim['Grupo de Operação'] == 'Preparo')]

agg_1_controle = {'QUANTIDADE': 'sum', 'DATA_OPERACAO':'max'}

id_controle_pre_plantio = id_controle_pre_plantio.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Característica','Grupo de Operação']).agg(agg_1_controle).reset_index()

id_controle_pre_plantio = id_controle_pre_plantio.merge(regua_operacao,right_on='operação', left_on='DCR_OPERACAO',how='left')

#Realizando filtros para ter somente insumos de Pré e que a regua de operação seja diferente de barra
id_controle_pre_plantio = id_controle_pre_plantio[(id_controle_pre_plantio['Característica']=='Pré')]
id_controle_pre_plantio = id_controle_pre_plantio[(id_controle_pre_plantio['descrição_regua'] !='Barra')]


#Importante definir o que é Pré e Pós preparo
id_controle_pre_plantio = id_controle_pre_plantio.merge(data_de_preparo, left_on='cd_uso_solo', right_on='cd_uso_solo', how='left').drop('Status', axis=1)
id_controle_pre_plantio['Pré-Pós Preparo'] = id_controle_pre_plantio.apply(lambda x: None if pd.isna(x['Data de Preparo']) else "Pós Preparo" if  x['DATA_OPERACAO'] >= x['Data de Preparo']  else "Pré Preparo", axis=1)

#Importante definir o que é Pré e Pós plantio
id_controle_pre_plantio = id_controle_pre_plantio.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']],left_on='cd_uso_solo', right_on='cd_uso_solo',how='left')
id_controle_pre_plantio['Pré-Pós Plantio'] = id_controle_pre_plantio.apply(lambda x: None if pd.isna(x['Data de Plantio']) else "Pós Plantio" if  x['DATA_OPERACAO'] >= x['Data de Plantio']  else "Pré Plantio", axis=1)


id_controle_pre_plantio = id_controle_pre_plantio[(id_controle_pre_plantio['Pré-Pós Preparo'] == 'Pós Preparo')]

#Removendo boletins duplicados para poder ordená-los e ter apenas um id por boletim
id_controle_pre_plantio = id_controle_pre_plantio.drop_duplicates('CD_BOLETIM_SILVICULTURA')

id_controle_pre_plantio = id_controle_pre_plantio.sort_values(['cd_uso_solo','DATA_OPERACAO'],ascending=[True,True])


#Trazendo os insumos que são Fordor
id_controle_pre_plantio = id_controle_pre_plantio.merge(insumos_pre_plantio[['CD_BOLETIM_SILVICULTURA','Fordor']], left_on='CD_BOLETIM_SILVICULTURA', right_on='CD_BOLETIM_SILVICULTURA',how='left')


numero_de_dias_apos_plantio = 5


#A operação precisa ser Pós preparo, e que seja 1º Pré com insumos que tenham características de Pré

id_controle_pre_plantio['gatilho_database'] = id_controle_pre_plantio.apply(lambda x: None if pd.isna(x['Data de Plantio']) else  
                                                        "1º Pré com insumo pre" if x['Pré-Pós Plantio'] == "Pós Plantio" and (x['DATA_OPERACAO'] - x['Data de Plantio']).days <= numero_de_dias_apos_plantio and x['Característica'] == 'Pré' else 
                                                        'Data Forçada' if x['DATA_OPERACAO'] > x['Data de Plantio'] and x['Grupo de Operação'] == 'Preparo' else 
                                                        "Pós Plantio" if x['DATA_OPERACAO'] > x['Data de Plantio'] else 
                                                        None , axis=1)





#Tudo que for diferente de pós plantio é selecionado
id_controle_pre_plantio = id_controle_pre_plantio[id_controle_pre_plantio['gatilho_database'] != 'Pós Plantio']

#Criando a regra para analisar a ordem das operações
id_controle_pre_plantio['Regra de mudança'] = (id_controle_pre_plantio['cd_uso_solo'] != id_controle_pre_plantio['cd_uso_solo'].shift(1)) & (id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA'] != id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA'].shift(1))

id_controle_pre_plantio['grupos_distintos'] = id_controle_pre_plantio['Regra de mudança'].cumsum()

id_controle_pre_plantio['Sequência de Operação'] = id_controle_pre_plantio.groupby('grupos_distintos').cumcount()+1

#Necessário considerar apenas o primeiro apontamento referente a operação
id_controle_pre_plantio = id_controle_pre_plantio[id_controle_pre_plantio['Sequência de Operação'] == 1]

#Criando referência do boletim
id_controle_pre_plantio['Boletim 1º pre'] = id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA']

id_controle_pre_plantio = id_controle_pre_plantio.rename({'DATA_OPERACAO':'Data 1º Pre', 'Sequência de Operação':'Pre 1º Operação'},axis=1)

id_controle_pre_plantio.to_excel('id_controle_pre_plantio.xlsx')


In [511]:
#ID 2º Controle

id_controle_pre = boletim[boletim['Grupo de Operação'] == 'Herbicida']


#Regra para agrupar. Lembrando que mesmo a data sendo Máxima, agrupo por boletim, então a data sempre será a do boletim.
agg_2_controle = {'QUANTIDADE':'sum', 'DATA_OPERACAO': 'max'}

id_controle_pre = id_controle_pre.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Característica']).agg(agg_2_controle).reset_index()

id_controle_pre = id_controle_pre.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']] , right_on='cd_uso_solo', left_on='cd_uso_solo', how='left')

id_controle_pre['Pré-Pós Plantio'] = id_controle_pre.apply(lambda x: None if pd.isna(x['Data de Plantio']) else "Pós Plantio" if  x['DATA_OPERACAO'] >= x['Data de Plantio']  else "Pré Plantio", axis=1)

id_controle_pre = id_controle_pre.merge(regua_operacao, right_on='operação', left_on='DCR_OPERACAO', how='left').drop('operação',axis=1)

id_controle_pre = id_controle_pre.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])


#Filtrar somente o que é Pós plantio

#1º A operação é pós plantio? Preciso filtrar apenas o que é pós
id_controle_pre = id_controle_pre[id_controle_pre['Pré-Pós Plantio'] == "Pós Plantio"]

#Removendo tudo o que eu já tenho boletim no 1ºControle Pré
id_controle_pre = id_controle_pre[ ~ id_controle_pre['CD_BOLETIM_SILVICULTURA'].isin(id_controle_pre_plantio['CD_BOLETIM_SILVICULTURA'])]

#2º Se a operação tiver o Nome de 2º Controle -> Devo Zerar a operação, sendo assim só posso trazer o que é 1º Controle ou o que é apontado como pré 
regua_id_2_controle_pre = (id_controle_pre['descrição_regua'] == 'Prés')

id_controle_pre = id_controle_pre[regua_id_2_controle_pre]


#3º Tem insumo de Pré? garantir que  a operação tenha sido apontada de forma correta, que ela tenha insumos de pré

#Mudança de Premissas -> Todas as operações pre emergentes deverão ser contadas e só devemos utilizar as duas primeiras -> A operação não dependerá apenas do nome dela agora

insumos_pre = (id_controle_pre['Característica'] == "Pré")

id_controle_pre = id_controle_pre[insumos_pre]

id_controle_pre = id_controle_pre.drop_duplicates('CD_BOLETIM_SILVICULTURA')

id_controle_pre = id_controle_pre.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

#Definindo uma regra de operações para a mudança de uma talhão para o outro
id_controle_pre['Regra de Mudança'] = (id_controle_pre['CD_BOLETIM_SILVICULTURA'] != id_controle_pre['CD_BOLETIM_SILVICULTURA'].shift(1)) & (id_controle_pre['cd_uso_solo'] != id_controle_pre['cd_uso_solo'].shift(1))

# Criar uma coluna para indicar grupos distintos
id_controle_pre['grupos_distintos'] = id_controle_pre['Regra de Mudança'].cumsum()

#Aplica a soma acumulada para cada grupo distinto
id_controle_pre['Sequência de Operação'] = id_controle_pre.groupby('grupos_distintos').cumcount()+1

#Base de sequencia de operações completa ->Solicitação Gabi
base_sequencia_controle_pre = id_controle_pre.copy()

#Filtrando o que tem somente até segunda operação, o restante das operações deverá ser desconsiderado
id_controle_pre = id_controle_pre[id_controle_pre['Sequência de Operação'] <= 2]

#Ajustando a ordem de operação {1:2,2:3}
id_controle_pre['Sequência de Operação'] = id_controle_pre['Sequência de Operação']+1 



#Inserir ajuste de Sequencia
id_controle_pre = id_controle_pre.merge(sequencia_vigente[['boletim','Operação Anterior', 'Operação Atual','Sequência Vigente']], left_on='CD_BOLETIM_SILVICULTURA' , right_on='boletim', how='left').drop('boletim',axis=1)
#Quando a sequência vigente não for nula, ela assume a nova sequência de operação, caso contrário ela segue vida
id_controle_pre['Sequência de Operação'] = id_controle_pre.apply(lambda x: x['Sequência Vigente'] if pd.notna(x['Sequência Vigente']) else x['Sequência de Operação'], axis=1)



#Fazendo Referência no boletim 
id_controle_pre['Boletim pre'] = id_controle_pre['CD_BOLETIM_SILVICULTURA']


id_controle_pre = id_controle_pre.rename({'DATA_OPERACAO':'Data Pre','Sequência de Operação':'Pre'},axis=1)


In [512]:
id_controle_pre.to_excel('id_controle_pre.xlsx')

In [513]:
#Não posso utilizar boletins repetidos dentro das outras operações -> Boletins dos Controles pre

#Não posso utilizar boletins repetidos dentro das outras operações -> Boletins dos Controles pre plantio

#Na barra não terá boletins repetidos devido aos filtros de Pós Plantio e Característica de Pós

In [514]:
#Controle de barra 


barra = boletim[boletim['Grupo de Operação'] == 'Herbicida']

agg_barra = {'QUANTIDADE': 'sum', 'DATA_OPERACAO':'max'}

#Boletim já é filtrado para operação e insumo vigente e apontamentos que não são cancelados e reprovados

barra = barra.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Grupo de Operação','Operação Vigente']).agg(agg_barra).reset_index()

barra = barra.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')

barra = barra.merge(insumos[['Insumo','Insumo Vigente','Característica pre e pos','Característica']], left_on='NOME_RECURSO', right_on='Insumo', how='left')

barra = barra.merge(regua_operacao,left_on='DCR_OPERACAO', right_on='operação',how='left').drop('operação',axis=1)

barra['Pré-Pós Plantio'] = barra.apply(lambda x: None if pd.isna(x['Data de Plantio']) else "Pós Plantio" if  x['DATA_OPERACAO'] >= x['Data de Plantio']  else "Pré Plantio", axis=1)



#Filtrando somente operação que é após plantio 
barra = barra[barra['Pré-Pós Plantio'] == 'Pós Plantio']

#Filtrando característica do insumo # Ou Pré e Pós
#Ajustar na planilha de premissas os herbicidas para duplicarem ->Impacto de duplicar a base
barra = barra[barra['Característica'] == 'Pós']

#Definindo coluna talhão para filtrar os 300
barra['Talhão'] = barra['objeto de locação'].str[-3:]

#Filtrando a descrição da operação e talhões 300, que podem ser talhões manuais
barra = barra[(barra['descrição_regua'] == 'Barra') | (barra['Talhão'] == '300')]

#Se a operação de barra for maior do que x dias após o plantio, não considerar
dias_barra_e_plantio = 420
barra['dias_barra_plantio'] = barra.apply(lambda x:  x['DATA_OPERACAO'] - x['Data de Plantio'] if pd.notna(x['DATA_OPERACAO']) else None, axis=1)
barra = barra[barra['dias_barra_plantio'] <= pd.Timedelta(days=dias_barra_e_plantio)]


#Remover duplicatas de boletim, devido a presença de mais de um insumo por boletim
barra = barra.drop_duplicates('CD_BOLETIM_SILVICULTURA')

#Ordenar por objeto de locação e por data
barra = barra.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

barra['Regra de mudança'] = (barra['cd_uso_solo'] != barra['cd_uso_solo'].shift(1)) & (barra['CD_BOLETIM_SILVICULTURA'] != barra['CD_BOLETIM_SILVICULTURA'].shift(1))

barra['grupos_distintos'] = barra['Regra de mudança'].cumsum()

barra['Sequência de Operação'] = barra.groupby('grupos_distintos').cumcount()+1

#Filtrar somente as três primeiras operações (1º, 2º e 3º  Barra)
barra= barra[barra['Sequência de Operação'] <=3]


#Inserir ajuste de Sequencia

barra = barra.merge(sequencia_vigente[['boletim','Operação Anterior', 'Operação Atual','Sequência Vigente']], left_on='CD_BOLETIM_SILVICULTURA' , right_on='boletim', how='left').drop('boletim',axis=1)


#Quando a sequência vigente não for nula, ela assume a nova sequência de operação, caso contrário ela segue vida
barra['Sequência de Operação'] = barra.apply(lambda x: x['Sequência Vigente'] if pd.notna(x['Sequência Vigente']) else x['Sequência de Operação'], axis=1)


#Fazendo Referência no boletim 
barra['Boletim Barra'] = barra['CD_BOLETIM_SILVICULTURA']

#Renomeando as colunas
barra = barra.rename({'DATA_OPERACAO':'Data Barra','Sequência de Operação':'Barra'},axis=1)


In [515]:
barra.to_excel("barra.xlsx")

In [516]:
#Calcário

#insumos maior que zero;
calcario = boletim[boletim['QUANTIDADE'] > 0]

#Filtrando o boletim apenas para o grupo de operações de adubação
calcario = boletim[boletim['Grupo de Operação'] == 'Calcário']


#Regra para agrupar. Lembrando que mesmo a data sendo Máxima, agrupo por boletim, então a data sempre será a do boletim.
agg_adub = {'QUANTIDADE':'sum', 'DATA_OPERACAO': 'max'}

calcario = calcario.groupby(['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','objeto de locação','cd_uso_solo','NOME_RECURSO','Grupo de Operação','Operação Vigente']).agg(agg_adub).reset_index()

#Merge para a área do talhão utilizando o cd_uso_solo, assim garante a área que já foi apontada no talhão original
calcario = calcario.merge(cadastro_ciclo[['cd_uso_solo','vlr_area']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')

calcario['dose/ha'] = calcario['QUANTIDADE'] / calcario['vlr_area']


calcario = calcario.merge(insumos[['Insumo','Característica','Insumo Vigente','Formulação']],left_on='NOME_RECURSO',right_on='Insumo',how='left').drop('Insumo',axis=1)

#Apenas características de calcario
calcario = calcario[(calcario['Característica'] == 'Calcário') | (calcario['Característica'] == 'Oxyfertil')]

#Apenas Insumo vigente

calcario = calcario[calcario['Insumo Vigente'] == 'Sim']

calcario = calcario.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']], left_on='cd_uso_solo', right_on='cd_uso_solo',how='left')

calcario['delta_operacao'] = (pd.to_datetime(calcario['DATA_OPERACAO']) - pd.to_datetime(calcario['Data de Plantio'])).dt.days

#A operação de calcário pode acontecer até 420 dias depois do plantio e um ano antes, e a complementar somente após



min_delta_calcario = -365
max_delta_calcario = 365

calcario = calcario[(calcario['delta_operacao'] <= max_delta_calcario) & (calcario['delta_operacao'] >= min_delta_calcario)]



#Necessário criar um DF auxiliar, devido ao insumos duplicados dentro de um mesmo boletim
calcario_regra_de_mudanca = calcario.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA')
#Ordenar o data frame

calcario_regra_de_mudanca = calcario_regra_de_mudanca.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

#Realizando a regra de mudança de cd_uso_solo e boletim
calcario_regra_de_mudanca['regra de mudança'] = (calcario_regra_de_mudanca['cd_uso_solo'] != calcario_regra_de_mudanca['cd_uso_solo'].shift(1)) & (calcario_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'] != calcario_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'].shift(1))
#Realizando a soma acumulada
calcario_regra_de_mudanca['grupos_distintos'] = calcario_regra_de_mudanca['regra de mudança'].cumsum()
#Criando a ordem de operação
calcario_regra_de_mudanca['Sequencia de Operação'] = calcario_regra_de_mudanca.groupby('grupos_distintos').cumcount()+1
#Trazendo a ordem da operação para adubação novamente
calcario = calcario.merge(calcario_regra_de_mudanca[['Sequencia de Operação','CD_BOLETIM_SILVICULTURA']], left_on='CD_BOLETIM_SILVICULTURA', right_on='CD_BOLETIM_SILVICULTURA', how='left')

#Somando o volume aplicado 
calcario['Dose/ha acumulada calagem'] = calcario.groupby('cd_uso_solo')['dose/ha'].transform('sum')


#Definir se a operação é pré ou pós plantio
calcario['status_calagem'] = calcario.apply(lambda x: 'Pós Plantio' if x['DATA_OPERACAO'] >= x['Data de Plantio'] else 'Pré Plantio', axis=1)

#Número de avaliações realizadas no mesmo cd_uso solo
calcario['numero_avaliacoes_calagem'] = calcario.groupby('cd_uso_solo')['cd_uso_solo'].transform('count')

#Contagem de status_calagem por operação apontados por cd_uso_solo
calcario['vlr_status_calagem'] = calcario.groupby(['cd_uso_solo','status_calagem'])['cd_uso_solo'].transform('count')


#Ordenar novamente o data frame de calcário para criar uma regra de filtrar as operações
calcario = calcario.sort_values(['cd_uso_solo','DATA_OPERACAO'], ascending=[True,True])

#Contagem de operações realizadas dentro do cd_uso_solo sendo pre_pos_calagem
calcario['pre_pos_calagem'] = calcario.groupby('cd_uso_solo')['status_calagem'].transform('nunique')

#Trazendo recomendação de calcario
calcario = calcario.merge(recomendacao_adu[['Recomendação de Calagem','Recomendação Dose kg/ha Calagem','Data de recomendação','Objeto de locação']], left_on='objeto de locação', right_on='Objeto de locação',how='left').drop('Objeto de locação',axis=1).rename({'Data de recomendação':'Data de recomendação calagem'},axis=1)


#gerando regra para considerar a operação 


calcario['regra_operacao_calagem'] = np.where(
    calcario['numero_avaliacoes_calagem'] == 1, 'Considerar 1 op',
    np.where(
        (calcario['status_calagem'] == 'Pré Plantio') &
        (calcario['numero_avaliacoes_calagem'] >= 2) &
        (calcario['numero_avaliacoes_calagem'] == calcario['Sequencia de Operação']),
        'considerar pre',
        np.where(
            (calcario['status_calagem'] == 'Pós Plantio') & 
            (calcario['numero_avaliacoes_calagem'] >= 2) & 
            (calcario['Sequencia de Operação'] == 1),
            'considerar pos',
            np.where(
                (calcario['pre_pos_calagem'] == 2) &
                (calcario['status_calagem'] == 'Pré Plantio') &
                (calcario['vlr_status_calagem'] == calcario['Sequencia de Operação']),
                'considerar pre max sequencia',
                'Não considerar'
            )
        )
    )
)


calcario.to_excel('calcario.xlsx')




#Filtrando somente as operações para considerar
calcario = calcario[calcario['regra_operacao_calagem'] != 'Não considerar']

calcario['Boletim Calagem'] = calcario['CD_BOLETIM_SILVICULTURA']

calcario = calcario.rename({'DATA_OPERACAO':'Data Calagem', 'NOME_RECURSO': 'Insumo calagem' , 'numero_avaliacoes_calagem': 'Número de Avaliações','Sequencia de Operação':'Sequência Calagem', 'regra_operacao':'regra_calagem'},axis=1)



In [517]:
#Adubação de base

#Filtrando apontamento de insumo maior que 0
base = boletim[boletim['QUANTIDADE'] > 0]

#Filtrando grupo de operação (Preparo e Adubação) e operação vigente
base = base[((base['Grupo de Operação'] == 'Adubação')| (base['Grupo de Operação'] == 'Preparo') ) & (base['Operação Vigente'] == 'Sim')] 

#Filtrando a característica e insumo vigente
base = base[((base['Característica'] == 'Adubo') | (base['Característica'] == 'Oxyfertil')) & (base['Insumo Vigente'] == 'Sim')]

#Filtrando o tipo de operação de adubação -> Base
base = base[(base['Tipo de Adubação'] == 'Base') | (base['Tipo de Adubação'] == 'Calagem')  | (base['Tipo de Adubação'] == 'Manutenção') ]

#Delta plantio em relação a operação
base['Delta de plantio'] = (base['DATA_OPERACAO'] - base['Data de Plantio']).dt.days

#Filtrando apenas o que poderia estar dentro do range máximo e mínimo de operação

#min = 90 e max = 120
base_min = -90
base_max = 150
base = base[(base['Delta de plantio'] > base_min) & (base['Delta de plantio'] < base_max)]

#Classificar o data frame por 'cd_uso_solo' e data de operação e a quantidade, assim garante que o insumo válido será sempre o de maior volume
base = base.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[True,True,False])


#retirar os boletins de calagem
base = base[ ~ base['CD_BOLETIM_SILVICULTURA'].isin(calcario['CD_BOLETIM_SILVICULTURA'])]


#Criar uma cópia do boletim para excluir os boletins duplicados, e gerar a ordem de operação 1 e 2, com isso trazer devolta para a operação de base
base_regra_de_mudanca = base.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA', keep='first')

base_regra_de_mudanca['regra_de_mudanca'] = (base_regra_de_mudanca['cd_uso_solo'] != base_regra_de_mudanca['cd_uso_solo'].shift(1)) & (base_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'] != base_regra_de_mudanca['CD_BOLETIM_SILVICULTURA'].shift(1))
#Realizando a soma acumulada
base_regra_de_mudanca['grupos_distintos'] = base_regra_de_mudanca['regra_de_mudanca'].cumsum()
#Criando a ordem de operação
base_regra_de_mudanca['Sequência base'] = base_regra_de_mudanca.groupby('grupos_distintos').cumcount()+1

#trazer para a base a sequencia de operação
base = base.merge(base_regra_de_mudanca[['CD_BOLETIM_SILVICULTURA','NOME_RECURSO','Sequência base']], left_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], right_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], how='left')

#Dose aplicada
base['Dose/ha base'] = base['QUANTIDADE'] / base['vlr_area']

#Dose aplicada acumulada
base['Dose/ha acumulada base'] = base.groupby(['cd_uso_solo'])['Dose/ha base'].transform('sum')


#Classificar o que é pré e pós plantio 
#Definir ordem para o que é pré e pós plantio

base['status_base'] = base.apply(lambda x: 'Pós Plantio' if x['DATA_OPERACAO'] >= x['Data de Plantio'] else 'Pré Plantio',axis=1)

#Encontrando o número de operações para o talhão
base['numero de avaliacoes base'] = base.groupby(['cd_uso_solo'])['cd_uso_solo'].transform('count')

#Contagem de status apontado por 'cd_uso_solo'
base['vlr_status_base'] = base.groupby(['cd_uso_solo','status_base'])['cd_uso_solo'].transform('count')

#Contagem distinta do número de status em um 'cd_uso_solo'
base['pre_pos_base'] = base.groupby('cd_uso_solo')['status_base'].transform('nunique')


#Trazendo a recomendação 

base = base.merge(recomendacao_adu[['Insumo base','Dose kg/ha base','Data de recomendação','Objeto de locação']], left_on='objeto de locação', right_on='Objeto de locação', how='left').drop('Objeto de locação',axis=1).rename({'Data de recomendação':'Data de recomendação base', 'Insumo base':'Insumo base - rec','Dose kg/ha base':'Dose kg/ha base - rec'},axis=1)

#Definindo valor aplicado para reforço
volume_reforço = 120


data_reforco = 60


#Se já tem base tem apontamento de adubação até 60 dias ele é reforço


#Gerando regra para considerar a operação

base['regra_operacao_base'] = np.where((base['numero de avaliacoes base'] == 1) & 
                                       (base['Sequência base'] ==1) , 
                                       'Considerar 1 Op  & > insumo',
                                       np.where((base['status_base'] == 'Pré Plantio') &
                                                (base['numero de avaliacoes base'] >= 2)&
                                                (base['numero de avaliacoes base'] == base['Sequência base']),'Considerar Pré',
                                                np.where((base['status_base']== 'Pós Plantio')&
                                                         (base['numero de avaliacoes base']>=2) &
                                                         (base['Sequência base'] ==1), 'Considerar Pós',
                                                         np.where((base['pre_pos_base'] == 2)&
                                                                  (base['status_base']== 'Pré Plantio')&
                                                                  (base['vlr_status_base'] == base['Sequência base']), 'considerar pre max sequencia',
                                                                  np.where( (base['Sequência base'] >= 2)&
                                                                            (base['status_base'] == 'Pós Plantio')&
                                                                            (base['Formulação'] == 'Oxyfertil') , 'Reforço',        
                                                                        np.where((base['Sequência base'] == 2) &
                                                                                (base['status_base'] == 'Pós Plantio')&
                                                                                (base['Dose/ha base'] <= volume_reforço), 'Reforço', 
                                                                                np.where(
                                                                                    (base['Sequência base'] >=2)&
                                                                                    (base['status_base'] == 'Pós Plantio')&
                                                                                    (base['Delta de plantio'] <= data_reforco), 'Reforço2',
                                                                                        'Não considerar'
                                                                                         )                                                                                                                                       
                                                                                )
                                                                        )                                                               
                                                                  )
                                                        )
                                                )
                                        )

#Exportar base bruta com todas as informações para chegar no resultado
base.to_excel('base.xlsx')

#Filtrar apenas operações que devem ser consideradas ao levar a informação para o boletim

base = base[(base['regra_operacao_base'] != 'Não considerar') & (base['regra_operacao_base'] != 'Reforço2') ]

#Fazendo Referência no boletim 
base['Boletim Base'] = base['CD_BOLETIM_SILVICULTURA']


base = base.rename({'DATA_OPERACAO':'Data Adub. Base','Estratégia':'Estratégia Base', 'NOME_RECURSO':'Insumo Adubação base' },axis=1)

#Falta deninir a operação de reforço
#criar uma lógica para cob1 e cob2 não ter os id's que já considerei na base



In [518]:
#Encontrando a estratégia de adubação

estrategia = base[['cd_uso_solo','Estratégia Base','QUANTIDADE','regra_operacao_base','Insumo Adubação base']]

#O que define a estratégia é o insumo e a sua quantidade determina qual será a do talhão
estrategia = estrategia.sort_values(['cd_uso_solo','QUANTIDADE'], ascending=[True,False])

estrategia = estrategia.drop_duplicates('cd_uso_solo',keep='first')

estrategia.to_excel('estrategia.xlsx')


#Quando não tenho apontamentos de base o que define a estratégia é a recomendação de base



In [519]:
#Adubações de cobertura 


adu_cob = boletim[boletim['QUANTIDADE'] >0]


#Filtro de grupo de operação -> Adubação e Operação vigente -> Sim
adu_cob = adu_cob[(adu_cob['Grupo de Operação']=='Adubação') & (adu_cob['Operação Vigente'] == 'Sim')]

#Filtrando grupo de insumos -> Caracteristica Adubo e Insumo vigente ->
adu_cob = adu_cob[((adu_cob['Característica'] == 'Adubo') | (adu_cob['Característica'] == 'Oxyfertil') ) & (adu_cob['Insumo Vigente'] == 'Sim')]

#Retirar os apontamentos de base em adu_cob -> Assim garante que não teremos apontamentos duplicados
adu_cob = adu_cob[ ~ adu_cob['CD_BOLETIM_SILVICULTURA'].isin(base['CD_BOLETIM_SILVICULTURA'])]

#Retirar os apontamentos de calagem em adu_cob 
adu_cob = adu_cob[ ~ adu_cob['CD_BOLETIM_SILVICULTURA'].isin(calcario['CD_BOLETIM_SILVICULTURA'])]


#Trazendo estratégia executada pela operação
adu_cob = adu_cob.merge(estrategia[['cd_uso_solo','Estratégia Base']], left_on='cd_uso_solo', right_on='cd_uso_solo' , how='left')


#Trazendo a estratégia recomendada pelo P&D
adu_cob = adu_cob.merge(recomendacao_adu[['Objeto de locação','Estratégia Recomendada', 'Insumo cobertura','Dose kg/ha cobertura','Insumo cobertura 2', 'Dose kg/ha cobertura 2','Data de recomendação']], left_on='objeto de locação', right_on='Objeto de locação', how='left').drop('Objeto de locação', axis=1).rename({'Insumo cobertura':'Insumo cobertura - rec','Dose kg/ha cobertura':'Dose kg/ha cobertura - rec', 'Insumo cobertura 2':'Insumo cobertura 2 - rec','Dose kg/ha cobertura 2':'Dose kg/ha cobertura 2 - rec', 'Data de recomendação': 'Data de recomendação cobertura'},axis=1)


#Trazendo a recomendação de complementar para a base
adu_cob = adu_cob.merge(recomendacao_comp[['Objeto de locação','NPK complementar','Dose kg/ha complementar','Corretivos complementares','Dose kg/ha corretivos','Data de recomendação']], left_on='objeto de locação', right_on='Objeto de locação', how='left').drop('Objeto de locação', axis=1).rename({'Dose kg/ha corretivos':'Dose kg/ha corretivos - rec','Dose kg/ha complementar': 'Dose kg/ha complementar - rec'},axis=1)


#Estratégia ajustada

adu_cob['Estratégia ajustada'] = adu_cob.apply(lambda x: x['Estratégia Recomendada'] if pd.isna(x['Estratégia Base']) else x['Estratégia Base'],axis=1)


#Definindo o status se é pré ou pós plantio
adu_cob['Status adu_cob'] = adu_cob.apply( lambda x: 'Pós Plantio' if x['DATA_OPERACAO'] >= x['Data de Plantio'] else None if pd.isna(x['Data de Plantio']) else 'Pré Plantio', axis=1)


#Coluna de delta de plantio
adu_cob['Delta de plantio adu_cob'] = (adu_cob['DATA_OPERACAO'] - adu_cob['Data de Plantio']).dt.days


#Filtrar somente o que é pós plantio
adu_cob = adu_cob[adu_cob['Status adu_cob'] == 'Pós Plantio']

adu_cob = adu_cob.sort_values(['cd_uso_solo','DATA_OPERACAO','QUANTIDADE'], ascending=[True,True,False])


#Dose aplicada
adu_cob['Dose/ha adu cob'] = adu_cob['QUANTIDADE'] / adu_cob['vlr_area']

#Dose aplicada acumulada
adu_cob['Dose/ha acumulada adu cob'] = adu_cob.groupby(['cd_uso_solo'])['Dose/ha adu cob'].transform('sum')


#Concaternar todos os insumos e formulações de um boletim para ter todos em uma mesma lista
adu_cob['Insumos apontados'] = adu_cob.groupby(['CD_BOLETIM_SILVICULTURA', 'DATA_OPERACAO',])['NOME_RECURSO'].transform(lambda x: ', '.join(x))
adu_cob['Formulações apontadas'] = adu_cob.groupby(['CD_BOLETIM_SILVICULTURA', 'DATA_OPERACAO',])['Formulação'].transform(lambda x: ', '.join(x))


#Ordernar o data frame -> tenho mais de um insumo apontado duplicando o boletim, preciso criar um dataframe intermediário
#O insumo que teve mais apontamento será o considerado para a estratégia
adu_cob_regra_mudanca = adu_cob.copy().drop_duplicates('CD_BOLETIM_SILVICULTURA', keep='first')
adu_cob_regra_mudanca['regra de mudança adu_cob'] = (adu_cob_regra_mudanca['cd_uso_solo'] != adu_cob_regra_mudanca['cd_uso_solo'].shift(1)) & (adu_cob_regra_mudanca['CD_BOLETIM_SILVICULTURA'] != adu_cob_regra_mudanca['CD_BOLETIM_SILVICULTURA'].shift(1))
adu_cob_regra_mudanca['grupos distindos adu_cob'] = adu_cob_regra_mudanca['regra de mudança adu_cob'].cumsum()
#Sequência de operação
adu_cob_regra_mudanca['Sequência Cob'] = adu_cob_regra_mudanca.groupby('grupos distindos adu_cob').cumcount()+1

#Removendo as duplicatas de adubação de cobertura para não duplicar ao levar para o boletim
#adu_cob = adu_cob.drop_duplicates('CD_BOLETIM_SILVICULTURA', keep='first')

#Trazendo a sequencia de operações  da base de adubação de cobertura regra de mudança
adu_cob = adu_cob.merge(adu_cob_regra_mudanca[['CD_BOLETIM_SILVICULTURA','NOME_RECURSO','Sequência Cob']], left_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], right_on=['CD_BOLETIM_SILVICULTURA','NOME_RECURSO'], how='left')

#Se a data de recomendação for maior que a data de operação, não devo considerar que houve recomendação
adu_cob['Existe recomendação'] = adu_cob.apply(lambda x: "Não" if ((x['Corretivos complementares'] == 'n.a.') | pd.isna(x['Corretivos complementares'])) & 
                                              ((x['NPK complementar'] == 'n.a.') | pd.isna(x['NPK complementar'])) |
                                              (x['DATA_OPERACAO'] < x['Data de recomendação'])
                                              else 'Sim',axis=1)


adu_cob['recomendação comp'] = adu_cob.apply(lambda x: "Sem recomendação" if ((x['Corretivos complementares'] == 'n.a.') | pd.isna(x['Corretivos complementares'])) & 
                                              ((x['NPK complementar'] == 'n.a.') | pd.isna(x['NPK complementar'])) else "Aderente" 
                                              if (x['NPK complementar'] in x['Formulações apontadas'].split(', ')) |
                                             (x['Corretivos complementares'] in x['Formulações apontadas'].split(', ')) else 'Não Aderente',
                                               axis=1)

delta_complementar = 450

adu_cob['regra_operação'] = np.where(adu_cob['Sequência Cob'] >= 5, "Desconsiderar",
                                    np.where(adu_cob['Sequência Cob'] == 1, 'Cob 1',
                                        np.where(
                                            (adu_cob['Estratégia ajustada'] == 'B') & 
                                            (adu_cob['Sequência Cob'] == 2), 'Cob 2',
                                                np.where( (adu_cob['Estratégia ajustada'] == 'B') &
                                                        ( adu_cob['Sequência Cob'] >2) & 
                                                        (adu_cob['Existe recomendação'] == 'Sim')&
                                                        (adu_cob['recomendação comp'] == 'Aderente'), 'Complementar',
                                                    np.where(
                                                        (adu_cob['Estratégia ajustada'] == 'B')& 
                                                        (adu_cob['Sequência Cob'] > 2) &
                                                        (adu_cob['Delta de plantio adu_cob'] >= delta_complementar), 'Complementar',
                                                        np.where(
                                                            (adu_cob['Estratégia ajustada'] == 'A')&
                                                            (adu_cob['Sequência Cob'] == 2)&
                                                            (adu_cob['Delta de plantio adu_cob'] < delta_complementar) &
                                                            (adu_cob['Existe recomendação'] == 'Não'), 'Cob 2',
                                                            np.where(
                                                                (adu_cob['Estratégia ajustada'] == 'A') &
                                                                (adu_cob['Sequência Cob'] >= 2) &
                                                                (adu_cob['Delta de plantio adu_cob'] >= delta_complementar), 'Complementar',
                                                                np.where(
                                                                    (adu_cob['Estratégia ajustada'] == 'A')&
                                                                    (adu_cob['Sequência Cob'] >= 2) &
                                                                    (adu_cob['Delta de plantio adu_cob'] < delta_complementar) &
                                                                    (adu_cob['recomendação comp'] == 'Aderente'), 'Complementar',
                                                                    np.where(
                                                                        (adu_cob['Estratégia ajustada'] == 'A')&
                                                                        (adu_cob['Sequência Cob'] >= 2)&
                                                                        (adu_cob['Delta de plantio adu_cob'] > delta_complementar) , 'Complementar',

                                                                        np.where(
                                                                            (adu_cob['Estratégia ajustada'] == 'A') &
                                                                            (adu_cob['Sequência Cob'] > 2) &
                                                                            (adu_cob['Delta de plantio adu_cob'] < delta_complementar),'Complementar',
                                                                            
                                                                            np.where(
                                                                                (adu_cob['Estratégia ajustada'] == 'T') &
                                                                                (adu_cob['Sequência Cob'] > 2), 'Complementar T', None
                                                                                        )
                                                                                    )
                                                                                )
                                                                            )
                                                                        )
                                                                    ) 
                                                                )
                                                            )
                                                        ) 
                                                    )
                                                )
                                        

adu_cob['Ordem complementar'] = adu_cob[adu_cob['regra_operação'] == 'Complementar'].groupby(['cd_uso_solo','regra_operação'])['cd_uso_solo'].cumcount()+1

adu_cob['regra_operação'] = adu_cob.apply(lambda x: str(x['regra_operação']) + ' ' + str(int(x['Ordem complementar'])) if (x['regra_operação'] == 'Complementar') else x['regra_operação'], axis=1)


#Exportando a base bruta
adu_cob.to_excel('adu_cob.xlsx')



#Fazendo Referência no boletim 
adu_cob['Boletim Adu cob'] = adu_cob['CD_BOLETIM_SILVICULTURA']

#Filtrando os dados para seguir somente o que realmente será necessário
adu_cob = adu_cob[(adu_cob['regra_operação'] != 'Desconsiderar') & ((adu_cob['Ordem complementar'].isna()) | (adu_cob['Ordem complementar'] <= 2)) & (pd.notna(adu_cob['regra_operação']))]


adu_cob = adu_cob.rename({'DATA_OPERACAO':'Data Adub. cob', 'NOME_RECURSO':'Insumo Cob' },axis=1)


In [520]:
#Replantio

#Trazer para o boletim as informações de Replantio
boletim = boletim.merge(replantio[['CD_BOLETIM_SILVICULTURA','Boletim Replantio','DCR_OPERACAO','NOME_RECURSO','cd_uso_solo','Data Replantio','regra_operacao_replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo','numero de replantios']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], how='left')


#Definir as colunas de 1º replantio e de 2º Replantio 
boletim['Data 1º Replantio'] = boletim.apply(lambda x : x['Data Replantio'] if x['regra_operacao_replantio'] == '1º Replantio' else None , axis=1)
boletim['1º Replantio'] = boletim.apply(lambda x : str(x['cd_uso_solo']) + str(x['regra_operacao_replantio']) if x['regra_operacao_replantio'] == '1º Replantio' else None , axis=1)



#Definir as colunas de 2º Replantio 
boletim['Data 2º Replantio'] = boletim.apply(lambda x : x['Data Replantio'] if x['regra_operacao_replantio'] == '2º Replantio' else None , axis=1)
boletim['2º Replantio'] = boletim.apply(lambda x : str(x['cd_uso_solo']) + str(x['regra_operacao_replantio']) if x['regra_operacao_replantio'] == '2º Replantio' else None , axis=1)


#Definir as colunas de 3º Replantio
boletim['Data 3º Replantio'] = boletim.apply(lambda x : x['Data Replantio'] if x['regra_operacao_replantio'] == '3º Replantio' else None , axis=1)
boletim['3º Replantio'] = boletim.apply(lambda x :  str(x['cd_uso_solo']) + str(x['regra_operacao_replantio']) if x['regra_operacao_replantio'] == '3º Replantio' else None , axis=1)

#Definir a coluna de apoio para replantio
boletim['Replantio_apoio'] = boletim.apply(lambda x :  str(x['cd_uso_solo']) + 'replantio' if (x['regra_operacao_replantio'] == '1º Replantio') | (x['regra_operacao_replantio'] == '2º Replantio') | (x['regra_operacao_replantio'] == '3º Replantio') else None , axis=1)



boletim.head(4)



,ID_REGIAO,CD_BOLETIM_SILVICULTURA,DATA_OPERACAO,VLR_PRODUCAO,DCR_OPERACAO,CD_TALHAO,ID_PROJETO,CD_PROJETO,NOME_RECURSO,QUANTIDADE,...,mudas/ha recomendado,Replantio Efetivo,numero de replantios,Data 1º Replantio,1º Replantio,Data 2º Replantio,2º Replantio,Data 3º Replantio,3º Replantio,Replantio_apoio
0,1.0,282232.0,2023-03-21,15.50,SERV CAP QUIM MANUAL LEVE AGRIC,001,0001,1.0,HERBICIDA ROUNDUP 20L,35.00,...,NaN,NaN,NaN,NaT,None,NaT,None,NaT,None,None
1,1.0,278364.0,2023-02-21,53.14,SERV CAP QUIM MEC 2ª BARRA AGRIC,001,0001,1.0,HERBICIDA POS EMERG GLI UP 720 WG 5KG,98.29,...,NaN,NaN,NaN,NaT,None,NaT,None,NaT,None,None
2,1.0,278364.0,2023-02-21,53.14,SERV CAP QUIM MEC 2ª BARRA AGRIC,001,0001,1.0,HERBICIDA PRE EMERGENTE ESPLANADE 1L,5.89,...,NaN,NaN,NaN,NaT,None,NaT,None,NaT,None,None
3,1.0,240564.0,2022-12-01,54.99,SERV ADUBACAO SOLIDA MEC AGRIC,001,0001,1.0,ADUBO 500 KG 14-00-28,16500.00,...,NaN,NaN,NaN,NaT,None,NaT,None,NaT,None,None


In [521]:
#1º Controle Pre emergente

#Trazer para o boletim no mesmo formato que trouxemos o 2º e o 3º controle
boletim = boletim.merge(id_controle_pre_plantio[['CD_BOLETIM_SILVICULTURA','Boletim 1º pre','DCR_OPERACAO','NOME_RECURSO','cd_uso_solo','Data 1º Pre','Pre 1º Operação']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], how='left')


#Transformando a coluna 1º Pre em número inteiro
boletim['1º Pre'] = pd.to_numeric(boletim['Pre 1º Operação'],errors='coerce').astype('Int64')

#Definindo o ID do 1º Pre
boletim['1º Pre'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['1º Pre'])+'ºControle') if pd.notna(x['1º Pre']) and x['1º Pre'] ==1 else None, axis=1) 

In [522]:
#2º e 3º Controle Pre emergente após plantio


boletim = boletim.merge(id_controle_pre[['CD_BOLETIM_SILVICULTURA','Boletim pre', 'cd_uso_solo','NOME_RECURSO','Pre','Data Pre']], left_on=['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'], right_on=['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'],how='left')

#Definindo a data do 2º Pré e o controle

boletim['Data 2º Pre'] = boletim.apply(lambda x: x['Data Pre'] if x['Pre'] == 2 else None,axis=1)

boletim['2º Pre'] = pd.to_numeric(boletim['Pre'],errors='coerce').astype('Int64')

boletim['2º Pre'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['2º Pre'])+'ºControle') if pd.notna(x['2º Pre']) and x['2º Pre'] ==2 else None, axis=1) 

#Definindo a data do 3º Pré e o controle

boletim['3º Pre'] = pd.to_numeric(boletim['Pre'],errors='coerce').astype('Int64')

boletim['Data 3º Pre'] = boletim.apply(lambda x: x['Data Pre'] if x['Pre'] == 3 else None, axis= 1)

boletim['3º Pre'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['3º Pre'])+'ºControle') if pd.notna(x['3º Pre']) and x['3º Pre'] ==3 else None, axis=1)


#Organizando as datas do Pre
boletim['Pre'] = boletim.apply(lambda x: x['Pre 1º Operação'] if pd.isna(x['Pre']) else x['Pre'], axis=1)
boletim['Data Pre'] = boletim.apply(lambda x: x['Data 1º Pre'] if pd.isna(x['Data Pre']) else x['Data Pre'], axis=1)

In [523]:
#Barra 

boletim = boletim.merge(barra[['CD_BOLETIM_SILVICULTURA','cd_uso_solo','Boletim Barra','NOME_RECURSO','Barra','Data Barra']], left_on=['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'], right_on= ['CD_BOLETIM_SILVICULTURA','cd_uso_solo','NOME_RECURSO'], how='left')

#Definindo a data da 1º Barra

boletim['Data 1º Barra'] = boletim.apply(lambda x: x['Data Barra'] if x['Barra'] == 1 else None,axis=1)

boletim['1º Barra'] = pd.to_numeric(boletim['Barra'],errors='coerce').astype('Int64')

boletim['1º Barra'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['1º Barra'])+'ºControle') if pd.notna(x['1º Barra']) and x['1º Barra'] ==1 else None, axis=1) 


#Definindo a data da 2º Barra

boletim['Data 2º Barra'] = boletim.apply(lambda x: x['Data Barra'] if x['Barra'] == 2 else None,axis=1)

boletim['2º Barra'] = pd.to_numeric(boletim['Barra'],errors='coerce').astype('Int64')

boletim['2º Barra'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['2º Barra'])+'ºControle') if pd.notna(x['2º Barra']) and x['2º Barra'] ==2 else None, axis=1) 


#Definindo a data da 3º Barra

boletim['Data 3º Barra'] = boletim.apply(lambda x: x['Data Barra'] if x['Barra'] == 3 else None,axis=1)

boletim['3º Barra'] = pd.to_numeric(boletim['Barra'],errors='coerce').astype('Int64')

boletim['3º Barra'] = boletim.apply(lambda x: (str(x['cd_uso_solo'])+ str(x['3º Barra'])+'ºControle') if pd.notna(x['3º Barra']) and x['3º Barra'] ==3 else None, axis=1) 





In [524]:
# Calagem

#Trazer a operação de calagem para o boletim
boletim = boletim.merge(calcario[['CD_BOLETIM_SILVICULTURA','Boletim Calagem','DCR_OPERACAO','Insumo calagem','cd_uso_solo','Sequência Calagem','Data Calagem','Dose/ha acumulada calagem','regra_operacao_calagem','Recomendação de Calagem','Recomendação Dose kg/ha Calagem','Data de recomendação calagem']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Insumo calagem'],how='left')

boletim['Calagem'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + 'Calagem' if pd.notna(x['Data Calagem']) else None, axis=1)


In [525]:
# Adubação de base

#Trazer as operações de adubação de base para o boletim

boletim = boletim.merge(base[['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Boletim Base','Insumo Adubação base','cd_uso_solo','Sequência base','Data Adub. Base','Dose/ha acumulada base','Dose/ha base','regra_operacao_base','Estratégia Base','Insumo base - rec','Dose kg/ha base - rec','Data de recomendação base']] ,left_on=['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO','cd_uso_solo'],right_on=['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Insumo Adubação base','cd_uso_solo'], how='left')

boletim['Adubação de base'] = boletim.apply(
    lambda x: str(x['cd_uso_solo']) + 'Adubação de base' if pd.notna(x['Data Adub. Base']) and x['regra_operacao_base'] != 'Reforço' else
              str(x['cd_uso_solo']) + 'Reforço' if pd.notna(x['Data Adub. Base']) and x['regra_operacao_base'] == 'Reforço' else
              None, axis=1)

In [526]:
#Adubação de cobertura

#trazer a operação de adubação para o boletirm

boletim = boletim.merge(adu_cob[['CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Boletim Adu cob','cd_uso_solo','Data Adub. cob','Estratégia ajustada','Insumos apontados', 'Dose/ha adu cob','Dose/ha acumulada adu cob','recomendação comp','Sequência Cob','regra_operação','Insumo Cob','Insumo cobertura - rec', 'Dose kg/ha cobertura - rec','Insumo cobertura 2 - rec', 'Dose kg/ha cobertura 2 - rec','Data de recomendação cobertura','NPK complementar','Dose kg/ha complementar - rec','Corretivos complementares','Dose kg/ha corretivos - rec']], left_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','NOME_RECURSO'], right_on=['cd_uso_solo','CD_BOLETIM_SILVICULTURA','DCR_OPERACAO','Insumo Cob'],how='left').rename({'NPK complementar':'NPK complementar - rec'},axis=1)

boletim['Adu cob'] = boletim.apply(lambda x: str(x['cd_uso_solo']) + str(x['regra_operação']) if pd.notna(x['Data Adub. cob']) else None, axis = 1)

In [527]:
boletim.to_excel('boletim.xlsx')

In [528]:
#Aderência 


aderencia = boletim[['cd_uso_solo','num_ciclo','cd_regime','num_rotacao','objeto de locação','nom_projeto','vlr_area','ID_REGIAO']]

aderencia = aderencia.drop_duplicates('cd_uso_solo')


pre_1 = "1ºControle"
pre_2 = "2ºControle"
pre_3 = "3ºControle"
barra_1 = "1ºControle"
barra_2 = "2ºControle"
barra_3 = "3ºControle"
calagem = "Calagem"
reforco = "Reforço"
adub_base = "Adubação de base"
cob1 = "Cob 1"
cob2 = "Cob 2"
complementar1 = "Complementar 1"
complementar2 = "Complementar 2"
replantio1 = '1º Replantio'
replantio2 = '2º Replantio'
replantio3 = '3º Replantio'
replantio_apoio = 'replantio'



#Criando os Ids para realizar o merge
aderencia['pre_1'] = aderencia['cd_uso_solo'].astype(str) + pre_1
aderencia['pre_2'] = aderencia['cd_uso_solo'].astype(str) + pre_2
aderencia['pre_3'] = aderencia['cd_uso_solo'].astype(str) + pre_3
aderencia['barra_1'] = aderencia['cd_uso_solo'].astype(str) + barra_1
aderencia['barra_2'] = aderencia['cd_uso_solo'].astype(str) + barra_2
aderencia['barra_3'] = aderencia['cd_uso_solo'].astype(str) + barra_3
aderencia['Calagem'] = aderencia['cd_uso_solo'].astype(str)+ calagem
aderencia['adub_base'] = aderencia['cd_uso_solo'].astype(str)+ adub_base
aderencia['adub_base_reforco'] = aderencia['cd_uso_solo'].astype(str)+ reforco
aderencia['Cob1'] = aderencia['cd_uso_solo'].astype(str)+cob1
aderencia['Cob2'] = aderencia['cd_uso_solo'].astype(str)+cob2
aderencia['Complementar 1'] = aderencia['cd_uso_solo'].astype(str)+complementar1
aderencia['Complementar 2'] = aderencia['cd_uso_solo'].astype(str)+complementar2
aderencia['replantio1'] = aderencia['cd_uso_solo'].astype(str) + replantio1
aderencia['replantio2'] = aderencia['cd_uso_solo'].astype(str) + replantio2
aderencia['replantio3'] = aderencia['cd_uso_solo'].astype(str) + replantio3
aderencia['replantio_apoio'] = aderencia['cd_uso_solo'].astype(str) + replantio_apoio


#Preparo
aderencia = aderencia.merge(data_de_preparo[['Data de Preparo','cd_uso_solo']], left_on='cd_uso_solo', right_on='cd_uso_solo', how='left')
aderencia = aderencia.rename({'Data de Preparo':'Data Preparo referência' },axis=1)

#Plantio
aderencia = aderencia.merge(data_de_plantio[['Data de Plantio','cd_uso_solo']], left_on= 'cd_uso_solo', right_on='cd_uso_solo', how='left')


#1º Replantio 
aderencia = aderencia.merge(boletim[['Data 1º Replantio','DCR_OPERACAO','Boletim Replantio','1º Replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo']] , left_on = 'replantio1', right_on = '1º Replantio', how = 'left')
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação 1º Replantio'},axis=1)


#2º Replantio 
aderencia = aderencia.merge(boletim[['Data 2º Replantio','DCR_OPERACAO','Boletim Replantio','2º Replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo']] , left_on = 'replantio2', right_on = '2º Replantio', how = 'left', suffixes = ('', ' 2º Replantio'))
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação 2º Replantio'},axis=1)

#3º Replantio 
aderencia = aderencia.merge(boletim[['Data 3º Replantio','DCR_OPERACAO','Boletim Replantio','3º Replantio','mudas/ha','mudas/ha recomendado','Replantio Efetivo']] , left_on = 'replantio3', right_on = '3º Replantio', how = 'left', suffixes = ('', ' 3º Replantio'))

aderencia = aderencia.rename({'DCR_OPERACAO':'Operação 3º Replantio'},axis=1)


#Replantio apoio 
aderencia = aderencia.merge(boletim[['Replantio_apoio','numero de replantios']].drop_duplicates('Replantio_apoio'), left_on = 'replantio_apoio', right_on = 'Replantio_apoio', how = 'left')


#1º Pre
aderencia = aderencia.merge(boletim[['Data 1º Pre','DCR_OPERACAO','1º Pre','Boletim 1º pre']], left_on='pre_1', right_on='1º Pre',how='left').drop('1º Pre',axis=1)
aderencia = aderencia.rename({'DCR_OPERACAO': 'Operação 1º Pre'},axis=1)



#2º Pre
aderencia = aderencia.merge(boletim[['Data 2º Pre','DCR_OPERACAO','2º Pre','Boletim pre']], left_on='pre_2', right_on='2º Pre',how='left').drop('2º Pre',axis=1)
aderencia = aderencia.rename({'Boletim pre':'Boletim 2º Pre','DCR_OPERACAO': 'Operação 2º Pre'}, axis=1)

#3º Pre
aderencia = aderencia.merge(boletim[['Data 3º Pre','DCR_OPERACAO','3º Pre','Boletim pre']], left_on='pre_3', right_on='3º Pre',how='left').drop('3º Pre',axis=1)
aderencia = aderencia.rename({'Boletim pre':'Boletim 3º Pre','DCR_OPERACAO': 'Operação 3º Pre' }, axis=1)

#1º Barra
aderencia = aderencia.merge(boletim[['Data 1º Barra','DCR_OPERACAO','1º Barra' , 'Boletim Barra']], left_on= 'barra_1', right_on='1º Barra',how='left').drop('1º Barra',axis=1)
aderencia = aderencia.rename({'Boletim Barra':'Boletim 1º Barra','DCR_OPERACAO':'Operação 1º Barra' }, axis=1)

#2º Barra
aderencia = aderencia.merge(boletim[['Data 2º Barra','DCR_OPERACAO','2º Barra','Boletim Barra']], left_on= 'barra_2', right_on='2º Barra',how='left').drop('2º Barra',axis=1)
aderencia = aderencia.rename({'Boletim Barra':'Boletim 2º Barra','DCR_OPERACAO':'Operação 2º Barra'}, axis=1)

#3º Barra
aderencia = aderencia.merge(boletim[['Data 3º Barra','DCR_OPERACAO','3º Barra','Boletim Barra']], left_on= 'barra_3', right_on='3º Barra',how='left').drop('3º Barra',axis=1)
aderencia = aderencia.rename({'Boletim Barra':'Boletim 3º Barra','DCR_OPERACAO':'Operação 3º Barra'}, axis=1)

#Calagem
aderencia = aderencia.merge(boletim[['Data Calagem','DCR_OPERACAO','Boletim Calagem','Dose/ha acumulada calagem','Calagem','Insumo calagem','Recomendação de Calagem','Recomendação Dose kg/ha Calagem','Data de recomendação calagem']], left_on='Calagem', right_on='Calagem', how='left')
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação Calagem'},axis= 1)


#Adubação de base
aderencia = aderencia.merge(boletim[['Data Adub. Base','DCR_OPERACAO','Boletim Base','Dose/ha acumulada base', 'Adubação de base','Insumo Adubação base','Estratégia','Insumo base - rec','Dose kg/ha base - rec','Data de recomendação base']] ,right_on='Adubação de base', left_on='adub_base', how = 'left')
aderencia = aderencia.rename({'DCR_OPERACAO':'Operação Base'},axis=1)


#Adubação de reforço
aderencia = aderencia.merge(boletim[['Data Adub. Base','DCR_OPERACAO','Boletim Base','Dose/ha base', 'Adubação de base','Insumo Adubação base']] ,right_on='Adubação de base', left_on='adub_base_reforco', how = 'left', suffixes=('', ' - Reforço'))
aderencia= aderencia.rename({'Dose/ha base':'Dose/ha base - Reforço','DCR_OPERACAO':'Operação de Reforço'}, axis=1)



#Cob 1
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob', 'Adu cob','Insumo cobertura - rec','Dose kg/ha cobertura - rec','Data de recomendação cobertura']], right_on='Adu cob', left_on='Cob1', how= 'left').rename({'Data de recomendação cobertura':'Data de recomendação cobertura 1'},axis=1)
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Cob 1','Data Adub. cob':'Data Adub. cob - Cob 1','DCR_OPERACAO':'Operação Cob 1'}, axis=1)

#Cob 2
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob', 'Adu cob','Insumo cobertura 2 - rec','Dose kg/ha cobertura 2 - rec','Data de recomendação cobertura']], right_on='Adu cob', left_on='Cob2', how= 'left',suffixes=('', ' - Cob 2')).rename({'Data de recomendação cobertura':'Data de recomendação cobertura 2'},axis=1)
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Cob 2','Data Adub. cob':'Data Adub. cob - Cob 2' , 'DCR_OPERACAO':'Operação Cob 2'}, axis=1)

#Complementar 1
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob', 'NPK complementar - rec' ,'Dose kg/ha complementar - rec','recomendação comp', 'Adu cob']], right_on='Adu cob', left_on='Complementar 1', how= 'left',suffixes=('', ' - Complementar 1'))
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Complementar 1','Data Adub. cob':'Data Adub. cob - Complementar 1','DCR_OPERACAO':'Operação complementar 1'}, axis=1)

#Complementar 2
aderencia = aderencia.merge(boletim[['Data Adub. cob','DCR_OPERACAO','Boletim Adu cob','Insumos apontados','Dose/ha acumulada adu cob','Corretivos complementares','Dose kg/ha corretivos - rec', 'Adu cob']], right_on='Adu cob', left_on='Complementar 2', how= 'left',suffixes=('', ' - Complementar 2'))
aderencia= aderencia.rename({'Boletim Adu cob': 'Boletim Complementar 2','Data Adub. cob':'Data Adub. cob - Complementar 2', 'DCR_OPERACAO':'Operação complementar 2'}, axis=1)



#Renomear as operações
aderencia = aderencia.rename({'Data de Preparo':'Data Preparo referência', 
'mudas/ha': 'mudas/ha 1º Replantio', 
'mudas/ha recomendado' : 'mudas/ha recomendado 1º Replantio' ,
'Replantio Efetivo' : 'Replantio Efetivo 1º Replantio' , 
'Boletim Replantio': 'Boletim Replantio - 1º Replantio'},axis=1)




# Criar uma máscara para identificar linhas onde todas as colunas são 'NaN'
colunas_nulas= ['Data 1º Pre', 'Data 2º Pre', 'Data 3º Pre', 'Data 1º Barra', 'Data 2º Barra','Data 3º Barra', 'Data Calagem' , 'Data Adub. Base', 
'Data Adub. Base - Reforço', 'Data Adub. cob - Cob 1','Data Adub. cob - Cob 2','Data Adub. cob - Complementar 1','Data Adub. cob - Complementar 2','Data 1º Replantio' ,'Data 2º Replantio','Data 3º Replantio' ]

aderencia_pre_emergente_mascara = aderencia[colunas_nulas].isna().all(axis=1)

#Excluindo somente  as colunas que não são nulas
aderencia = aderencia[~aderencia_pre_emergente_mascara]


aderencia = aderencia.drop(['ID_REGIAO','pre_1','pre_2','pre_3','barra_1','barra_2','barra_3','Calagem','adub_base',
'Adubação de base','adub_base_reforco' , 'Adubação de base - Reforço','Adu cob','Adu cob - Cob 2','Adu cob - Complementar 1',
'Adu cob - Complementar 2','Cob1', 'Cob2','Complementar 1','Complementar 2','1º Replantio','2º Replantio','3º Replantio','replantio1','replantio2','replantio3','replantio_apoio','Replantio_apoio'],axis=1)

In [529]:
#Definindo mês operacional das operações
#aderencia = mes_operacional(aderencia,'Data de Preparo','Mês Operacional Preparo')
#aderencia = mes_operacional(aderencia,'Data de Plantio','Mês Operacional Plantio')
#aderencia = mes_operacional(aderencia,'Data 1º Pre','Mês Operacional 1º Pre')
#aderencia = mes_operacional(aderencia,'Data 2º Pre','Mês Operacional 2º Pre')
#aderencia = mes_operacional(aderencia,'Data 3º Pre','Mês Operacional 3º Pre')
#aderencia = mes_operacional(aderencia,'Data 1º Barra','Mês Operacional 1º Barra')
#aderencia = mes_operacional(aderencia,'Data 2º Barra','Mês Operacional 2º Barra')


In [530]:
aderencia.to_excel('aderencia.xlsx')

In [531]:
#Conferência se há talhões duplicados

dpct_aderencia = aderencia[aderencia.cd_uso_solo.duplicated()].shape[0]
hora_fim = datetime.today()
tempo_de_execucao = hora_fim - hora_inicio
last_note = boletim['DATA_OPERACAO'].max().strftime('%d/%m/%y')
hora_exe = datetime.now().strftime('%H:%M:%S')

print(f"""O último apontamento do boletim é de: {last_note};
Temos {dpct_aderencia} talhões duplicados na base de Aderência;
Executado com sucesso;
O tempo de execução foi de {round(tempo_de_execucao.total_seconds()/60)} minutos.
Executado as {hora_exe} hrs.
""")

O último apontamento do boletim é de: 31/05/24;
Temos 0 talhões duplicados na base de Aderência;
Executado com sucesso;
O tempo de execução foi de 2 minutos.
Executado as 10:14:46 hrs.

